# **Prediction of window use case using generic markov model**

## Section 1 : Imports

In [11]:
import generic_markov
from data_parsing import create_cluster_map, create_window_dataframe, parse_app_data

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure


USE_APP_DATA = False

## Section 2 : Data Generation (Optional)

This section is only used if no app data is available, random data is generated :
- Picking a coordinates using a biased distribution of gps areas
- Adding a time, picked randomly in [9:00 ; 10:10] or [17:00 ; 18:10]
- Each having their own probability of opening/closing the window

In [12]:
if USE_APP_DATA:
    import random
    import csv
    from random import randint
    from data_parsing import format_time,parse_csv


    RANGE = 100
    window_state = []
    days = [randint(0, 6) for x in range(RANGE)]
    adresses_polygon = [[43.575414, 1.364311, 43.575223, 1.364048],
                        [43.579395, 1.378290, 43.579204, 1.378027],
                        [43.597612, 1.433209, 43.597421, 1.432946],
                        [43.594434, 1.465131, 43.594243, 1.464868],
                        [43.583149, 1.450255, 43.582958, 1.449992]]

    for k in range(10):
            rand = np.random.choice(list(range(5)), RANGE, p=[0.05, 0.5, 0.1, 0.1, 0.25])
            adresses = [[random.uniform(adresses_polygon[i][2], adresses_polygon[i][0]),
                        random.uniform(adresses_polygon[i][3], adresses_polygon[i][1])]
                        for i in rand]

            Starting_hours = np.linspace(9, 10.15)
            Stopping_hours = np.linspace(17, 18.15)

            possible_times = np.concatenate([Starting_hours, Stopping_hours])
            random_times = np.random.choice(possible_times, RANGE)
            Time = []

            FMT = '%H:%M'
            for index in range(RANGE):
                Time.append(format_time(random_times[index]))
                if (rand[index] == 0 or rand[index] == 1):
                    window_state.append(np.random.choice([0, 1], 1, p=[0.8, 0.2]))
                else:
                    window_state.append(np.random.choice([0, 1], 1, p=[0.01, 0.99]))

            with open('/home/celadodc-rswl.com/corentin.tatger/PersoPdata/app_data/dummy_data_{}.csv'.format(k),
                    mode='w') as csv_file:
                fieldnames = ['Pos_lat', 'Pos_lon', 'Wd_state', 'Time', 'Day']
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

                writer.writeheader()
                for i in range(RANGE):
                    writer.writerow({'Pos_lat': adresses[i][0], 'Pos_lon': adresses[i][1], 'Wd_state': window_state[i][0],
                                    'Time': Time[i], 'Day': days[i]})
    df_csv = parse_csv("/home/celadodc-rswl.com/corentin.tatger/PersoPdata/app_data/")
    df_window = create_window_dataframe(df_csv)


## Section 3: App data parsing

If data from the android app is available, parses through it to generate and train the model.

In [ ]:
if not USE_APP_DATA:
    

## Section 4 : Display clusters on a map

To improve readability and make debugging easier, a function displaying the clusters has been implemented. We use it here to check the if the clusters created using our dataset are logical. We call this function with an option to display data points considered a noise by the clustering algorithm, again to check for errors